In [ ]:
import requests, json, os, sys, time, re
from datetime import datetime
from multiprocessing import Pool

In [ ]:
## split the long list for multiprocessing
def split_list(lst_long, n):
    lst_splitted = []
    if len(lst_long) % n == 0:
        totalBatches = int(len(lst_long) / n)
    else:
        totalBatches = int(len(lst_long) / n + 1)
    for i in range(totalBatches):
        lst_short = lst_long[i*n:(i+1)*n]
        lst_splitted.append(lst_short)
    return lst_splitted

## create work status bar
def show_work_status(singleCount, totalCount, currentCount=0):
    if totalCount > 0:
        currentCount += singleCount
        percentage = 100.0 * currentCount / totalCount
        status =  '>' * int(percentage)  + '-' * (100 - int(percentage))
        sys.stdout.write('\r[{0}] {1:.2f}% '.format(status, percentage))
        sys.stdout.flush()
        if percentage >= 100:
            print ('\n')

In [ ]:
## read user id one by one
path_user_id = 'data/steam_user_id.txt'
with open(path_user_id, 'r') as f:
    lst_user_id = f.readlines()

In [ ]:
def worker(lst_user_id_temp):
    dic_temp = {}
    for user_id in lst_user_id_temp:
        base_url = 'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/' #more info about GetOwnedGames - https://developer.valvesoftware.com/wiki/Steam_Web_API
        params = {
            'key' : 'D0C62157A8941F12A687382B6D635449', #get Steam web API key using a Steam developer account
            'steamid' : user_id.strip(),
            'format' : 'json' }
        r = requests.get(base_url, params = params)
        user_inventory = r.json().get('response').get('games')
        dic_temp.update({user_id.strip():user_inventory}) #user_inventory doesn't contain user_id, so combine user_id and user_inventory into a dictionary
        time.sleep(1)
    return dic_temp

In [ ]:
## multiprocessing
p = Pool(5)

total_count = len(lst_user_id)
current_count = 0
show_work_status(0, total_count, current_count)

dic_master = {}
for i in split_list(lst_user_id,100):
    lst_temp_dic = p.map(worker, split_list(i,10))
    for x in lst_temp_dic:
        dic_master.update(x)
    show_work_status(len(i), total_count, current_count)
    current_count += len(i)
    time.sleep(5)

In [ ]:
with open('data/user_inventory.txt', 'w') as f:
    for user_id, user_inventory in dic_master.items():
        f.write(json.dumps({user_id:user_inventory}))
        f.write('\n')